In [5]:
# Importing files
from neo4j import GraphDatabase
import pandas as pd


# Creating a class to generate KG 
class RetailGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def create_knowledge_graph(self, data):
        with self.driver.session() as session:
            session.write_transaction(self._create_constraints)
            for index, row in data.iterrows():
                session.write_transaction(self._create_and_link, row, index)

    # Creating a constraint mechanism 
    @staticmethod
    def _create_constraints(tx):
        tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (s:Stock) REQUIRE s.code IS UNIQUE")
        tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Customer) REQUIRE c.id IS UNIQUE")
        tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (country:Country) REQUIRE country.name IS UNIQUE")


    # Creating a linking mechanism 
    @staticmethod
    def _create_and_link(tx, row, index):
        if pd.isna(row['Customer ID']):
            return

        tx.run("""
        MERGE (stock:Stock {code: $stockCode})
        ON CREATE SET stock.description = $description
        MERGE (customer:Customer {id: $customerId})
        MERGE (country:Country {name: $country})
        CREATE (transaction:Transaction {
            index: $index,
            invoiceNumber: $invoice, 
            quantity: $quantity, 
            price: $price,
            invoiceDate: $invoiceDate})
        MERGE (transaction)-[:CONTAINS]->(stock)
        MERGE (transaction)-[:MADE_BY]->(customer)
        MERGE (customer)-[:LOCATED_IN]->(country)
        MERGE (stock)-[:POPULAR_IN]->(country)
        """, 
        index=index,
        invoice=row['Invoice'], 
        invoiceDate=str(row['Invoice Date']),
        stockCode=row['Stock Code'], 
        description=row['Description'],
        quantity=row['Quantity'], 
        price=row['Price'], 
        customerId=row['Customer ID'],  
        country=row['Country'])


# Reading the excel to DataFrame 
df = pd.read_excel(r'C:\Users\bhavi\Downloads\Ira\KG_dataset.xlsx')


# Establishing the Neo4j connection
retail_graph = RetailGraph(
    uri='neo4j+s://8d5cac00.databases.neo4j.io', 
    user="neo4j", 
    password="xMsJ4ErS-ctnozyZdZWe67hP19-6UgYgnXRgqp_p3kY"
)

# Creating graph
retail_graph.create_knowledge_graph(df)

# Closing connection
retail_graph.close()


C:\Users\bhavi\AppData\Local\Temp\ipykernel_5616\697284337.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_constraints)
C:\Users\bhavi\AppData\Local\Temp\ipykernel_5616\697284337.py:18: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(self._create_and_link, row, index)
Failed to read from defunct connection IPv4Address(('8d5cac00.databases.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:ab19', 7687, 0, 0)))
Transaction failed and will be retried in 1.0626630446354524s (Failed to read from defunct connection IPv4Address(('8d5cac00.databases.neo4j.io', 7687)) (ResolvedIPv6Address(('64:ff9b::227e:ab19', 7687, 0, 0))))
Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Unable to retrieve routing information
Transaction failed and will be retried in 2.2763597389106307s (Unable to retrieve